<a href="https://colab.research.google.com/github/Mariam-Elbishbeashy/ANA_AI_Models/blob/main/Questionnaire-models%20%26%20datasets/Questionnaire_arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import random
from collections import Counter

# ==================== ENHANCED CHARACTER DEFINITIONS (ARABIC) ====================
CHARACTERS_AR = [
    # 🛡️ المديرين (الحراس الاستباقيون)
    "الناقد", "الكمالي", "المُرضي", "المتحكّم",
    "البارد", "مدمن الشغل", "الحيران",

    # 🔥 رجال الإطفاء (الحراس التفاعليون)
    "المؤجِّل", "الآكل", "اللاعب",

    # 😥 المنفيين (الأجزاء الجريحة)
    "الوحيد", "الخايف", "المهمَل", "الخجول",
    "المرهَق", "المعتمد", "الغيور", "الطفل الجريح"
]

# Character archetype groups (ARABIC)
MANAGERS_AR = ["الناقد", "الكمالي", "المُرضي", "المتحكّم",
               "البارد", "مدمن الشغل", "الحيران"]
FIREFIGHTERS_AR = ["المؤجِّل", "الآكل", "اللاعب"]
EXILES_AR = ["الوحيد", "الخايف", "المهمَل", "الخجول",
             "المرهَق", "المعتمد", "الغيور", "الطفل الجريح"]

# English to Arabic mapping for reference
EN_TO_AR = {
    "Inner Critic": "الناقد",
    "Perfectionist": "الكمالي",
    "People Pleaser": "المُرضي",
    "Controller": "المتحكّم",
    "Stoic Part": "البارد",
    "Workaholic": "مدمن الشغل",
    "Confused Part": "الحيران",
    "Procrastinator": "المؤجِّل",
    "Overeater/Binger": "الآكل",
    "Excessive Gamer": "اللاعب",
    "Lonely Part": "الوحيد",
    "Fearful Part": "الخايف",
    "Neglected Part": "المهمَل",
    "Ashamed Part": "الخجول",
    "Overwhelmed Part": "المرهَق",
    "Dependent Part": "المعتمد",
    "Jealous Part": "الغيور",
    "Wounded Child": "الطفل الجريح"
}

AR_TO_EN = {v: k for k, v in EN_TO_AR.items()}

# ==================== ARABIC QUESTION MAPPINGS ====================
QUESTION_MAPPINGS_AR = {
    'Q1': {
        0: ["المتحكّم", "الكمالي"],        # أعمل قوائم وخطط
        1: ["مدمن الشغل", "البارد"],       # أضغط على نفسي وأخلص الحاجات
        2: ["المُرضي", "المعتمد"],         # أتأكد من الناس
        3: ["المؤجِّل", "اللاعب"],         # ألاقي حاجات ألهّي بيها دماغي
        4: ["البارد", "المتحكّم"],         # أقول لنفسي "أنا هقدر أتعامل مع الموضوع"
        5: ["الحيران", "الوحيد"]          # محتاج وقت هادي لوحدي
    },

    'Q2_slider': {
        '0-20%': [],  # جزء كمالي بسيط
        '21-50%': ["الكمالي"],  # في حاجات من الكمالية
        '51-80%': ["الكمالي", "الناقد"],  # جزء كمالي نشيط
        '81-100%': ["الكمالي", "الناقد", "مدمن الشغل"]  # جزء كمالي مسيطر
    },

    'Q3': {
        0: ["المتحكّم", "الكمالي"],        # بحاول أصلح أو أحلّ الشعور
        1: ["الآكل", "المؤجِّل"],          # بلاقي حاجة آكلها، أشوفها، أو أعملها
        2: ["الطفل الجريح", "الخايف"],     # بحس صغير، عيل، أو خايف
        3: ["الناقد", "البارد"],           # أقول لنفسي "مينفعش أحس كده"
        4: ["الحيران", "المهمَل"],         # أحس إني منفصل عن جسمي أو مشاعري
        5: ["الناقد", "الخجول"]            # بزعل من نفسي
    },

    'Q4_slider': {
        '0-20%': [],  # جزء وحيد بسيط
        '21-50%': ["الوحيد"],  # في حاجات من الوحدة
        '51-80%': ["الوحيد", "المعتمد", "الطفل الجريح"],  # جزء وحيد نشيط
        '81-100%': ["الوحيد", "المهمَل", "الطفل الجريح"]  # جزء وحيد مسيطر
    },

    'Q5': {
        0: ["الكمالي", "الناقد"],          # جزء دايماً بيدوّي
        1: ["الوحيد", "المهمَل"],          # جزء بيحس بوحدة شديدة
        2: ["المتحكّم", "الخايف"],         # جزء بيستخدم التحكم
        3: ["المؤجِّل", "اللاعب"],         # جزء بيتهرب للنشاطات
        4: ["الخجول", "الناقد"],           # جزء بيحس بالخزي
        5: ["الحيران", "الطفل الجريح"]     # جزء متحير
    },

    'Q6': {
        0: ["الناقد", "الكمالي"],          # "مش مشكلة تعمل غلطات"
        1: ["الخايف", "المعتمد"],          # "مش هسيبك"
        2: ["المُرضي", "الخجول"],          # "مش محتاج تاخد حب"
        3: ["المهمَل", "البارد"],          # "مشاعرك ليها قيمة"
        4: ["الخايف", "المرهَق"],          # "أنت في أمان دلوقتي"
        5: ["الخجول", "الطفل الجريح"]      # "أنا بقبل كل حاجة فيك"
    },

    'Q7': {
        0: ["مدمن الشغل", "المتحكّم"],     # بشتغل أكتر ولفترات أطول
        1: ["الآكل", "المؤجِّل"],          # بستنى على أكل مريح أو مسلسلات
        2: ["البارد", "المهمَل"],          # ببطل أحس بأي حاجة
        3: ["المُرضي", "المعتمد"],         # بحاول أرضي كل الناس
        4: ["اللاعب", "المؤجِّل"],         # بنسى نفسي في ألعاب الفيديو
        5: ["الناقد", "الخجول"]            # بانتقد نفسي
    },

    'Q8_slider': {
        '0-20%': [],  # نشاط جزء الإطفائي بسيط
        '21-50%': ["المؤجِّل", "الآكل"],  # في حاجات من المؤجل/الآكل
        '51-80%': ["المؤجِّل", "الآكل", "اللاعب"],  # أنماط جزء الإطفائي نشيطة
        '81-100%': ["المؤجِّل", "الآكل", "اللاعب"]  # إستراتيجية جزء الإطفائي مسيطرة
    },

    'Q9': {
        0: ["المُرضي", "المعتمد"],         # أضع حدود من غير ما أحس بالذنب
        1: ["الخايف", "الوحيد"],           # أثق إن الناس مهتمة بيا
        2: ["الغيور", "الخجول"],           # ما أقارنش نفسي بالغير
        3: ["المعتمد", "المهمَل"],         # أطلب إللي أنا محتاجه
        4: ["الوحيد", "الطفل الجريح"],     # أحس إني متشاف وفاهَم
        5: ["المعتمد", "الخايف"]           # أخليني أتكل على حد
    },

    'Q10': {
        0: ["المُرضي", "المعتمد"],         # "اتأكد إنهم يعجبوا بيك"
        1: ["البارد", "المتحكّم"],         # "ماتورّيش أي نقطة ضعف"
        2: ["الغيور", "الخجول"],           # "هما متحكمين في حياتهم أكتر مني"
        3: ["الغيور", "الناقد"],           # "يفضّل أبقى زيهم"
        4: ["الوحيد", "الحيران"],          # "أنا مش منتمي ليهم هنا"
        5: ["الخجول", "الخايف"]            # "هو لو شافوني إللي جوا"
    },

    'Q11': {
        0: ["الناقد", "الخجول"],           # حس مستمر إنك "مش كفاية"
        1: ["الوحيد", "المهمَل"],          # وحدة عميقة
        2: ["الخايف", "الكمالي"],          # قلق من إن كل حاجة ممكن تمشي غلط
        3: ["المهمَل", "الحيران"],         # خدر نفسي أو فراغ
        4: ["الخجول", "مدمن الشغل"],       # ذنب عشان مقدرتش أعمل/أبقى أكتر
        5: ["الحيران", "الطفل الجريح"]     # حيرة في إيه اللي أنا عايزه
    },

    'Q12': {
        0: ["الكمالي", "مدمن الشغل"],      # قد إيه بدوّي
        1: ["المهمَل", "الطفل الجريح"],    # قد إيه أنا متأذي جوا
        2: ["الخايف", "الكمالي"],          # قد إيه أنا خايف أفشل
        3: ["الوحيد", "المهمَل"],          # قد إيه الوحدة عندي قوية
        4: ["المعتمد", "المرهَق"],         # قد إيه أنا محتاج مساعدة
        5: ["الحيران", "الطفل الجريح"]     # قد إيه أنا متحيّر
    },

    'Q13': {
        0: ["المرهَق", "الخايف"],          # بحس إني مغمور
        1: ["الغيور", "الخجول"],           # ببدأ أقارن نفسي بالناس
        2: ["الكمالي", "المتحكّم"],        # ببقى مركز جامد على كل حاجة
        3: ["المعتمد", "الخجول"],          # بحس إني حمل على الناس
        4: ["اللاعب", "الحيران"],          # بتهرب لعالم خيالي
        5: ["الناقد", "البارد"],           # بزعل من نفسي
        6: ["المعتمد", "الطفل الجريح"],    # نفسي في حد ياخد عني كل المسؤولية
        7: ["المهمَل", "البارد"]           # بحس بخَدَر
    }
}

# Strong signature patterns for each character (ARABIC)
CHARACTER_SIGNATURES_AR = {
    "الكمالي": ["Q2:high", "Q1:0", "Q3:0", "Q5:0"],
    "الناقد": ["Q2:high", "Q3:3", "Q5:0", "Q11:0"],
    "المُرضي": ["Q10:0", "Q7:3", "Q9:0", "Q1:2"],
    "المتحكّم": ["Q1:0", "Q3:0", "Q10:1", "Q7:0"],
    "البارد": ["Q1:4", "Q3:3", "Q7:2", "Q13:7"],
    "مدمن الشغل": ["Q1:1", "Q7:0", "Q12:0", "Q2:high"],
    "الحيران": ["Q1:5", "Q3:4", "Q5:5", "Q11:5"],
    "المؤجِّل": ["Q8:high", "Q1:3", "Q3:1", "Q7:4"],
    "الآكل": ["Q8:high", "Q3:1", "Q7:1", "Q13:1"],
    "اللاعب": ["Q8:high", "Q1:3", "Q7:4", "Q13:4"],
    "الوحيد": ["Q4:high", "Q1:5", "Q11:1", "Q12:3"],
    "الخايف": ["Q11:2", "Q6:1", "Q9:1", "Q13:0"],
    "المهمَل": ["Q4:high", "Q3:4", "Q6:3", "Q11:3"],
    "الخجول": ["Q11:0", "Q6:5", "Q9:2", "Q10:5"],
    "المرهَق": ["Q13:0", "Q12:4", "Q6:4", "Q11:2"],
    "المعتمد": ["Q9:5", "Q12:4", "Q10:0", "Q6:1"],
    "الغيور": ["Q10:2", "Q9:2", "Q13:1", "Q10:3"],
    "الطفل الجريح": ["Q6:5", "Q12:1", "Q3:2", "Q9:4"]
}

def generate_probabilistic_dataset_arabic(num_samples=75000):
    """Generate Arabic dataset with probabilistic character presence"""

    print(f"توليد {num_samples:,} عينة احتمالية بالعربية...")

    dataset = []
    slider_values = ['0-20%', '21-50%', '51-80%', '81-100%']

    for i in range(num_samples):
        if i % 10000 == 0 and i > 0:
            print(f"  تم توليد {i:,} عينة...")

        responses = {}

        # ===== 1. GENERATE PRIMARY CHARACTER (CORE) =====
        primary_char = random.choice(CHARACTERS_AR)
        primary_archetype = None
        if primary_char in MANAGERS_AR:
            primary_archetype = "مدير"
        elif primary_char in FIREFIGHTERS_AR:
            primary_archetype = "رجل إطفاء"
        else:
            primary_archetype = "منفي"

        # ===== 2. GENERATE SECONDARY CHARACTERS =====
        secondary_chars = []
        tertiary_chars = []

        # Based on primary, likely secondary characters
        if primary_archetype == "مدير":
            candidate_secondaries = [c for c in CHARACTERS_AR if c != primary_char]
            # Prefer same archetype or related exiles
            secondary = random.choice([c for c in candidate_secondaries if
                                      (c in MANAGERS_AR) or
                                      (primary_char in ["الناقد", "الكمالي"] and c in ["الخجول", "الطفل الجريح"])])
            secondary_chars.append(secondary)

            # Tertiary: often an exile
            remaining = [c for c in EXILES_AR if c not in secondary_chars]
            if remaining:
                tertiary_chars.append(random.choice(remaining))

        elif primary_archetype == "رجل إطفاء":
            candidate_secondaries = [c for c in FIREFIGHTERS_AR if c != primary_char]
            if candidate_secondaries:
                secondary_chars.append(random.choice(candidate_secondaries))

            # Often have a manager that's too strict
            tertiary_chars.append(random.choice(["الناقد", "الكمالي", "المتحكّم"]))

        else:  # منفي
            if primary_char in ["الوحيد", "المهمَل"]:
                secondary_chars.append(random.choice(["المُرضي", "مدمن الشغل"]))
                tertiary_chars.append(random.choice(["البارد", "الحيران"]))
            elif primary_char in ["الخجول", "الطفل الجريح"]:
                secondary_chars.append(random.choice(["الناقد", "الكمالي"]))
                tertiary_chars.append("المؤجِّل")
            else:
                secondary_chars.append(random.choice(MANAGERS_AR + FIREFIGHTERS_AR))

        # Ensure we have 2-4 total characters
        all_chars = list(set([primary_char] + secondary_chars + tertiary_chars))
        if len(all_chars) > 4:
            all_chars = random.sample(all_chars, 4)
        elif len(all_chars) < 2:
            all_chars.append(random.choice([c for c in CHARACTERS_AR if c not in all_chars]))

        # ===== 3. GENERATE RESPONSES BASED ON CHARACTERS =====
        # Generate answers that strongly reflect primary character
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']:
            # Start with primary character's preferred answers
            primary_answers = []
            for ans, chars in QUESTION_MAPPINGS_AR.get(q, {}).items():
                if primary_char in chars:
                    primary_answers.append(ans)

            # Add secondary characters' answers
            secondary_answers = []
            for char in secondary_chars:
                for ans, chars in QUESTION_MAPPINGS_AR.get(q, {}).items():
                    if char in chars:
                        secondary_answers.append(ans)

            # Combine with weights: primary 60%, secondary 40%
            all_answers = []
            if primary_answers:
                n_primary = random.randint(1, min(2, len(primary_answers)))
                all_answers.extend(random.sample(primary_answers, n_primary))

            if secondary_answers:
                n_secondary = random.randint(0, min(1, len(secondary_answers)))
                if n_secondary > 0:
                    all_answers.extend(random.sample(secondary_answers, n_secondary))

            # Ensure 1-3 answers total
            if not all_answers:
                max_opt = 6 if q != 'Q13' else 8
                all_answers = random.sample(range(max_opt), random.randint(1, 2))

            if len(all_answers) > 3:
                all_answers = random.sample(all_answers, 3)

            responses[q] = ','.join(map(str, sorted(set(all_answers))))

        # Generate slider responses
        def get_slider_for_char(char, q_prefix):
            """Get slider value based on character"""
            if char == "الكمالي" and q_prefix == "Q2":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            elif char == "الوحيد" and q_prefix == "Q4":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            elif char in FIREFIGHTERS_AR and q_prefix == "Q8":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            else:
                return random.choice(slider_values)

        # Sliders reflect strongest character for that trait
        responses['Q2'] = get_slider_for_char(primary_char, "Q2")
        responses['Q4'] = get_slider_for_char(primary_char, "Q4")
        responses['Q8'] = get_slider_for_char(primary_char, "Q8")

        # ===== 4. CREATE PROBABILITY TARGETS =====
        for char in CHARACTERS_AR:
            col_name = f'prob_{char}'

            if char == primary_char:
                # Primary: high probability
                responses[col_name] = round(random.uniform(0.8, 0.98), 2)
            elif char in secondary_chars:
                # Secondary: medium-high probability
                responses[col_name] = round(random.uniform(0.4, 0.7), 2)
            elif char in tertiary_chars:
                # Tertiary: medium-low probability
                responses[col_name] = round(random.uniform(0.2, 0.5), 2)
            else:
                # Others: low probability, but not zero
                responses[col_name] = round(random.uniform(0.0, 0.2), 2)

        # ===== 5. ADD DERIVED FEATURES =====
        # Feature 1: Number of answers per question
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']:
            responses[f'{q}_count'] = len(responses[q].split(','))

        # Feature 2: Convert sliders to numeric
        slider_map = {'0-20%': 0.1, '21-50%': 0.35, '51-80%': 0.65, '81-100%': 0.9}
        responses['Q2_num'] = slider_map[responses['Q2']]
        responses['Q4_num'] = slider_map[responses['Q4']]
        responses['Q8_num'] = slider_map[responses['Q8']]

        # Feature 3: Archetype indicators
        responses['has_manager_indicators'] = int(
            responses['Q2_num'] > 0.6 or  # كمالية
            responses.get('Q1', '').split(',').count('0') > 0 or  # تحكم
            responses.get('Q10', '').split(',').count('0') > 0   # إرضاء الناس
        )

        responses['has_exile_indicators'] = int(
            responses['Q4_num'] > 0.6 or  # وحدة
            responses.get('Q11', '').split(',').count('1') > 0 or  # شعور بالوحدة
            responses.get('Q13', '').split(',').count('0') > 0    # إرهاق
        )

        responses['has_firefighter_indicators'] = int(
            responses['Q8_num'] > 0.6 or  # هروب
            responses.get('Q7', '').split(',').count('1') > 0 or  # أكل مريح
            responses.get('Q7', '').split(',').count('4') > 0    # ألعاب
        )

        dataset.append(responses)

    # Create DataFrame
    question_cols = [f'Q{i}' for i in range(1, 14)]
    derived_cols = [col for col in dataset[0].keys() if col.startswith('Q') and '_num' in col or '_count' in col or 'has_' in col]
    prob_cols = [f'prob_{char}' for char in CHARACTERS_AR]

    all_cols = question_cols + derived_cols + prob_cols
    df = pd.DataFrame(dataset)[all_cols]

    return df, CHARACTERS_AR

def create_top3_dataset_arabic(df, characters_ar):
    """Convert probabilistic dataset to top-3 format for training (Arabic)"""

    prob_cols = [f'prob_{char}' for char in characters_ar]
    prob_matrix = df[prob_cols].values

    # Get top 3 indices for each row
    top3_indices = np.argsort(prob_matrix, axis=1)[:, -3:][:, ::-1]

    # Create new dataset with top 3 characters as strings
    top3_data = []
    for idx, row in df.iterrows():
        row_data = {col: row[col] for col in df.columns if not col.startswith('prob_')}

        # Get top 3 characters and their probabilities
        top3_chars = []
        top3_probs = []

        for rank in range(3):
            char_idx = top3_indices[idx, rank]
            char_name = characters_ar[char_idx]
            prob = prob_matrix[idx, char_idx]

            row_data[f'top{rank+1}_char'] = char_name
            row_data[f'top{rank+1}_prob'] = round(prob, 2)

            top3_chars.append(char_name)
            top3_probs.append(prob)

        # Add confidence metrics
        row_data['confidence_score'] = round(np.mean(top3_probs), 2)
        row_data['margin_1_2'] = round(top3_probs[0] - top3_probs[1], 2)
        row_data['top3_list'] = ','.join(top3_chars)

        # Add English equivalents for reference
        row_data['top1_char_en'] = AR_TO_EN.get(top3_chars[0], top3_chars[0])
        row_data['top2_char_en'] = AR_TO_EN.get(top3_chars[1], top3_chars[1])
        row_data['top3_char_en'] = AR_TO_EN.get(top3_chars[2], top3_chars[2])

        top3_data.append(row_data)

    top3_df = pd.DataFrame(top3_data)
    return top3_df

def analyze_top3_dataset_arabic(df, characters_ar):
    """Analyze the Arabic top-3 dataset"""

    print("\n" + "="*60)
    print("تحليل مجموعة البيانات العربية (أفضل 3 شخصيات)")
    print("="*60)

    # Character appearance in top 3
    char_counts = {char: 0 for char in characters_ar}

    for i in range(1, 4):
        col = f'top{i}_char'
        counts = df[col].value_counts()
        for char, count in counts.items():
            char_counts[char] += count

    print("ظهور الشخصيات في المراكز الثلاثة الأولى:")
    print("-" * 50)

    sorted_chars = sorted(char_counts.items(), key=lambda x: x[1], reverse=True)
    for char, count in sorted_chars:
        percentage = (count / (len(df) * 3)) * 100
        print(f"{char:20} {count:6,} ({percentage:5.1f}% من المراكز الثلاثة الأولى)")

    # Confidence analysis
    print(f"\nإحصائيات الثقة:")
    print(f"متوسط الثقة: {df['confidence_score'].mean():.2f}")
    print(f"ثقة عالية (>0.7): {(df['confidence_score'] > 0.7).sum():,} عينة")
    print(f"ثقة متوسطة (0.4-0.7): {((df['confidence_score'] >= 0.4) & (df['confidence_score'] <= 0.7)).sum():,} عينة")
    print(f"ثقة منخفضة (<0.4): {(df['confidence_score'] < 0.4).sum():,} عينة")

    # Margin analysis
    print(f"\nالفرق بين المركز الأول والثاني:")
    print(f"متوسط الفرق: {df['margin_1_2'].mean():.2f}")
    print(f"فائز واضح (فرق > 0.2): {(df['margin_1_2'] > 0.2).sum():,} عينة")
    print(f"تنافس شديد (فرق < 0.1): {(df['margin_1_2'] < 0.1).sum():,} عينة")

    # Archetype distribution in top 1
    top1_archetypes = []
    for char in df['top1_char']:
        if char in MANAGERS_AR:
            top1_archetypes.append("مدير")
        elif char in FIREFIGHTERS_AR:
            top1_archetypes.append("رجل إطفاء")
        else:
            top1_archetypes.append("منفي")

    archetype_counts = pd.Series(top1_archetypes).value_counts()
    print(f"\nنوعية الشخصية في المركز الأول:")
    for archetype, count in archetype_counts.items():
        print(f"  {archetype}: {count:,} عينة ({count/len(df):.1%})")

def prepare_model_training_arabic(df):
    """Prepare features and targets for model training (Arabic)"""

    # Feature columns
    question_cols = [f'Q{i}' for i in range(1, 14)]
    derived_cols = [col for col in df.columns if col.endswith('_num') or col.endswith('_count') or ('has_' in col and 'indicator' in col)]
    feature_cols = question_cols + derived_cols

    # Target columns (top 3 characters)
    target_cols = ['top1_char', 'top2_char', 'top3_char']

    print(f"\nإعداد التدريب:")
    print(f"الميزات: {len(feature_cols)} عمود")
    print(f"العينات: {len(df):,}")
    print(f"الأهداف: {len(target_cols)} أعمدة (المراكز الثلاثة الأولى)")

    return df[feature_cols], df[target_cols], feature_cols

# ==================== GENERATE ARABIC DATASET ====================
print("="*60)
print("توليد مجموعة بيانات ANA المحسنة باللغة العربية للتنبؤ بأفضل 3 شخصيات")
print("="*60)

DATASET_SIZE = 100000  # مجموعة بيانات أكبر لتحسين التعلم
prob_df_ar, characters_ar = generate_probabilistic_dataset_arabic(DATASET_SIZE)

# Convert to top-3 format
top3_df_ar = create_top3_dataset_arabic(prob_df_ar, characters_ar)

# Save datasets
prob_df_ar.to_csv('ana_dataset_probabilistic_arabic.csv', index=False, encoding='utf-8-sig')
top3_df_ar.to_csv('ana_dataset_top3_arabic.csv', index=False, encoding='utf-8-sig')

print(f"\n✅ تم حفظ مجموعة البيانات الاحتمالية: 'ana_dataset_probabilistic_arabic.csv'")
print(f"✅ تم حفظ مجموعة بيانات أفضل 3 شخصيات: 'ana_dataset_top3_arabic.csv'")

# Analyze
analyze_top3_dataset_arabic(top3_df_ar, characters_ar)

# Prepare for model training
X_ar, y_ar, feature_cols_ar = prepare_model_training_arabic(top3_df_ar)

print(f"\n" + "="*60)
print("مجموعة البيانات جاهزة لتدريب النموذج!")
print("="*60)

# Display sample data
print("\n" + "="*60)
print("عينة من البيانات العربية:")
print("="*60)
print("\nالأسئلة الأولى (5 أسطر):")
print(top3_df_ar[['Q1', 'Q2', 'Q3', 'Q4', 'Q5']].head())

print("\nالشخصيات الثلاثة الأولى (5 أسطر):")
print(top3_df_ar[['top1_char', 'top1_prob', 'top2_char', 'top2_prob', 'top3_char', 'top3_prob']].head())

print("\nالتوزيع الأولي للشخصيات:")
print("-" * 40)
top1_counts = top3_df_ar['top1_char'].value_counts().head(10)
for char, count in top1_counts.items():
    print(f"{char:20}: {count:6,} ({count/len(top3_df_ar):.1%})")

توليد مجموعة بيانات ANA المحسنة باللغة العربية للتنبؤ بأفضل 3 شخصيات
توليد 100,000 عينة احتمالية بالعربية...
  تم توليد 10,000 عينة...
  تم توليد 20,000 عينة...
  تم توليد 30,000 عينة...
  تم توليد 40,000 عينة...
  تم توليد 50,000 عينة...
  تم توليد 60,000 عينة...
  تم توليد 70,000 عينة...
  تم توليد 80,000 عينة...
  تم توليد 90,000 عينة...

✅ تم حفظ مجموعة البيانات الاحتمالية: 'ana_dataset_probabilistic_arabic.csv'
✅ تم حفظ مجموعة بيانات أفضل 3 شخصيات: 'ana_dataset_top3_arabic.csv'

تحليل مجموعة البيانات العربية (أفضل 3 شخصيات)
ظهور الشخصيات في المراكز الثلاثة الأولى:
--------------------------------------------------
المؤجِّل             25,987 (  8.7% من المراكز الثلاثة الأولى)
الكمالي              25,524 (  8.5% من المراكز الثلاثة الأولى)
الناقد               25,495 (  8.5% من المراكز الثلاثة الأولى)
المُرضي              20,087 (  6.7% من المراكز الثلاثة الأولى)
الحيران              19,936 (  6.6% من المراكز الثلاثة الأولى)
المتحكّم             19,880 (  6.6% من المراكز الثلاثة الأ